In [12]:
import boto3
import os
import json
from dotenv import load_dotenv

In [13]:
# 認証情報を環境変数から読み込み
load_dotenv()


True

In [14]:
# 認証情報を直接指定
session = boto3.Session(
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)



In [15]:
region_name = 'us-east-1'
# ナレッジベースID
knowledge_base_id = os.getenv("knowledge_base_id")
# データソースID
data_source_id = os.getenv("data_source_id")

kb_client_runtime = session.client("bedrock-agent-runtime", region_name=region_name)
bedlock_runtime = session.client(service_name="bedrock-runtime", region_name=region_name)

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

In [16]:
# 推論用コード
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = "application/json"
    content_type = "application/json"

    response = bedlock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

# クエリ拡張用コード
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        print("拡張クエリ", q)
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    # ここでなぜか解答を真ん中2つに絞っている？　確かに解答前後は空白ではあるが？
    return generated_queries[1:-1]

In [17]:
# 検索クエリを生成する関数(こちらの方が拡張性がある？)
def generate_search_queries(question: str, n: int):
    """
    Google 検索エンジン用の検索クエリを生成する
    """
    GENERATE_QUERIES = """
        User question: {{question}}
        Format: {"queries": ["query_1", "query_2", "query_3"]}
        """

    response = bedlock_runtime.invoke_model(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1024,
                # "system": "You are an expert at generating search queries for the Google search engine. Generate two search queries that are relevant to this question in Japanese. Output only valid JSON.",
                "system": "以下に示すQueryはユーザの入力した検索クエリです。このクエリに関連するクエリを多角的な視点から{n}つ生成してください。Formatに従って結果を出力してください。出力は正しいJSON形式でお願いします",
        
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": GENERATE_QUERIES.replace(
                                    "{{question}}", question
                                ),
                            }
                        ],
                    },
                ],
                "temperature": 0,
            }
        ),
    )

    result = json.loads(response.get("body").read())
    search_queries = result["content"][0]["text"]
    search_queries = json.loads(search_queries)

    return search_queries


In [18]:
# RAG検索用コード
def kb_search(query, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    print(res)
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

In [19]:
# RRFのコード
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [9]:
# original_query = '確定申告について教えて'
original_query = '「長期」 「積立」 「分散」投資を組み合わせると…どうなりますか？'


queries = generate_queries(original_query)
queries.insert(0, original_query)
print(queries)

all_results = {}
for q in queries:
    all_results[q] = kb_search(q)

print(all_results)


拡張クエリ 
拡張クエリ 4: 確定申告の期限について
拡張クエリ 4: 確定申告書の書き方
拡張クエリ 4: 確定申告の控除項目
拡張クエリ 4: 確定申告の必要書類
拡張クエリ 
['確定申告について教えて', '確定申告書の書き方', '確定申告の控除項目']
{'ResponseMetadata': {'RequestId': 'ac709dcb-2c69-4358-b295-ddf68e901fd3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 May 2024 05:28:53 GMT', 'content-type': 'application/json', 'content-length': '27474', 'connection': 'keep-alive', 'x-amzn-requestid': 'ac709dcb-2c69-4358-b295-ddf68e901fd3'}, 'RetryAttempts': 0}, 'retrievalResults': [{'content': {'text': '55   56   57   58   そ   の   他   配偶者の合計所得金額   専従者給与（控除）額の合計額   青色申告特別控除額   （   申告書Ｂ   区 分医療費控除   ������� 税務署長 令和��年��月��日   30条）○印税 理 士 署名押印 電話番号   税理士法書面提出   自宅・勤務先・携帯 － －   電話 番号   ※記入不要 ※記入不要   ※記入不要 ※記入不要   記号部分（5桁） 番号部分（2桁〜8桁）   I ××× 0 −××××××××   第 一 表   公金受取口座の登録・利用 「還付される税金の受取場所」欄に記載した預貯金口座を公金受取口座として新たに登録する場合や既に登録してい る公金受取口座を変更する場合には、「公金受取口座登録の同意」欄に〇〇を記入します。既に登録済みの場合には、 再度「公金受取口座登録の同意」欄に〇〇の記入は不要です。 ※ 上記の方法のほか、マイナポータルからも公金受取口座の登録が可能です（https://myna.go.jp）。 ※ 公金受取口座の登録には時間がかかる場合があります。お

In [12]:
original_query = '確定申告について教えて'


queries2 = generate_search_queries(original_query, 4)
print(queries2)

{'queries': ['確定申告の手続きについて', '確定申告の期限と必要書類', '確定申告の税金計算方法']}


In [10]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_results).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'

prompt = """
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。

<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_query)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['.............................................................. 45 6.下書き用申告書 .............................................................................................................................................. 46   令和5年分の所得税等の確定申告の相談及び申告書の受付   令和6年2月16日（金） から 同年3月15日（金）まで 還付申告書は、令和6年２月15日（木）以前でも提出できます。   税務署の閉庁日（土・日曜・祝日等）は、通常、税務署での相談及び申告書の受付は行っておりません。   申告書の提出方法 ①e-Taxで申告する。   ②郵便又は信書便により、住所地等の所轄税務署又は業務センター（※）に送付する。 ※�一部の税務署では、複数の税務署の内部事務を専担部署（業務センター）で集約処理する「内部事務の センター化」を実施しています。対象の税務署に郵送で提出する場合は、業務センター宛に送付して ください。 \u3000なお、対象の税務署については、国税庁ホームページをご覧ください。 ○�確定申告書は、「信書」に当たることから、税務署又は業務センターに送付する場合には、「郵 便物」（第一種郵便物）又は「信書便物」として送付する必要があります（郵便物・信書便物以外 の荷物扱いで送付することはできません。）。   ○通信日付印を提出日とみなします。通信日付印が申告期限内となるよう、お早めにご送付ください。   ③住所地等の所轄税務署の受付に提出する。 税務署の時間外収受箱への投函により、提出することもできます。 なお、業務センターに直接持参する方法で提出することはできませんので、ご注意ください。   1.確定申告の概要   ◆収受日付印のある確定申告書の控えが必要な場合 ○郵便又は信書便による送付又は税務署の時間外収受箱への投函により申告書を提出する場合は、複写により作成 した（複写式でないものについては、ボールペンで記載した）申告書の控えのほか返信用封筒（宛名をご記入の上、 所要額の切手を貼付してください。）を同封してください。

In [11]:
print(invoke_claude(prompt, model_id))

確定申告に関しては、以下の点に注意が必要です。

1. 確定申告の手続き
- 申告期間は2月16日から3月15日までです。郵送またはe-Taxで提出できます。
- 申告書の控えに収受印をもらう場合は、郵送時に返信用封筒を同封するか、受付で控えを持参する必要があります。

2. 納税の方法
- 納付期限は3月15日です。キャッシュレス納付が便利です。
- 振替納税を利用する場合は3月15日までに手続きが必要です。

3. 控除の記入
- 医療費控除、社会保険料控除、生命保険料控除などの控除を受ける場合は、第一表と第二表に記入が必要です。
- 年末調整で控除を受けた分は、源泉徴収票を確認して第二表に記入します。

4. 住民税・事業税
- 別居家族の氏名・住所を記入する必要があります。
- 所得税と取扱いが異なる点があるので、「住民税・事業税に関する事項」を確認します。

以上の点に注意して、確定申告の手続きを進めましょう。不明な点があれば税務署に問い合わせることも大切です。
